# Домашнее задание № 1

## 1. Задание на удаление тегов (2 балла)

Часто в данных, с которыми нам нужно работать помимо текста присутствует ещё какая-то лишняя информация: тэги, ссылки, код, разметка. Она, конечно, не всегда лишняя, но обычно от неё лучше избавиться.

Возьмем в качестве примера статью с Хабрахабра. Она скачана автоматически и в там остались некоторые тэги.

In [1]:
import re
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem
from nltk.tokenize import word_tokenize

In [2]:
text = '''
Сегодняшняя заметка будет о сервисе отслеживания активности пользователя — <a href="http://www.crazyegg.com" title="Сумасшедшие яйца">CrazyEgg</a>. Я не знаю кому обязан сервис таким говорящим именем, но оно работает, и хорошо. Запоминается? Отлично!<br><br><img src="http://img172.imageshack.us/img172/8434/18274658kc4.png" alt="Сумасшедшее яйцо"><br><br><blockquote><h3>Что это такое?</h3><br>
Как уже сказано выше это сервис для отслеживания перемещения пользователей по сайту — кто куда кликнул, какие ссылки наиболее популярные и тому подобная, разнородная информация для юзабилистов.<br>
Сервис позволяет отслеживать активность определенных пользователей, и выводить эти данные в различных формах: «инфракрасная» — где чем активнее область, тем она «теплее», салюты (чем активнее область, тем больше конфети), простой список с сортировкой по активности, колбы (чем заполненнее колба, тем активнее область), облака (на мой взгляд наиболее удобный вариант — совмещает в себе все остальные вместе взятые).<br><br><img src="https://habrastorage.org/getpro/habr/post_images/7c7/298/33c/7c729833cd942cc493e68833e3e0f12d.jpg" alt="Тепловое отслеживание популярности"><br></blockquote><br><br><a name="habracut"></a><br><br><blockquote><h3>Для кого это?</h3><br>
Разумеется сервис изначально планировался как первый помощник для юзабилистов, но пользоваться им может каждый, у кого есть деньги — интуитивный интерфейс, и хороший дизайн дает о себе знать — сервис массово популяризировался пол года-год назад. Относительно недорогой, но уж точно не из дешевых — сервис предполагает собой 4 платных линии и одну бесплатную. Дабы написать этот обзор я не поленился заплатить 19 долларов (в месяц) выбрав средний вариант — для нескольких проектов с включенными дополнительными функциями.<br><br><img src="https://habrastorage.org/getpro/habr/post_images/0b0/0a0/b16/0b00a0b16b1eda28e35f39487dcd2545.jpg" alt="Отслеживание ссылок"></blockquote><br><blockquote><img src="https://habrastorage.org/getpro/habr/post_images/0b5/433/892/0b54338921665ffb5a90930147296f5b.jpg" alt="Список по популярности"><br><br>
Да-да, сервис не бесплатен. Точнее бесплатная возможность потестировать есть, но она немного обрезана (можно отслеживать только 5000 посетителей и всего 4 страницы на сайте (внимание — 4 страницы, а не сайтов), т.е. вполне хватает для того, чтобы понять полезность сервиса). Зачем это?</h3><br>
С помощью этого сервиса можно тасовать блоки на сайте, которые полезны пользователям больше всего. Больше не нужно спорить создателям — какой блок где расположить. В этом им поможет <a href="http://ttp://www.crazyegg.com" title="Сумасшедшие яйца">Сумасшедшее яйцо</a>.<br><br><img src="https://habrastorage.org/getpro/habr/post_images/38c/af8/e75/38caf8e753782a01dc6419d3902edd57.jpg" alt="Добавление проекта"></blockquote><br><blockquote>Также этот сервис поможет вам понять — в какой зоне сайта лучше всего располагать рекламу, когда в вашем сервисе речь зайдет о монетизации. Ведь альтруизм это хорошо, а деньги на содержание сервиса нужны, и не лишним будет вычислить зоны где реклама будет приносить наибольшую отдачу, и наименьшее раздражение у пользователей.<br><br><img src="https://habrastorage.org/getpro/habr/post_images/0dd/3fb/b34/0dd3fbb34709e77f3fbcd6523c7eac77.jpg" alt="еще полезности"></blockquote><br><blockquote><h3>Как это работает?</h3><br>
Никаких километровых скриптов вставлять не нужно, достаточно вставиь 2 строчки яваскрипта, и сервис начнет отслеживание. Насколько я понял — исполнительный скрипт работает на сервере <a href="http://www.crazyegg.com" title="Сумасшедшие яйца">CrazyEgg</a>, поэтому ваш сайт от этого в производительности не потеряет ни секунды, а полезность довольно таки большая.<br><br><img src="https://habrastorage.org/getpro/habr/post_images/a90/e4e/e67/a90e4ee6760978463e6306a2f5982e24.jpg" alt="управление популярностью"></blockquote><br><blockquote><h3>Сколько это стоит?</h3><br>
Как я уже упоминал — сервис далеко не бесплатен, хоть и имеет тестовую-бесплатную версию. Расскажу подробнее о тарификации.</blockquote><br><blockquote>1. Бесплатная тестовая версия. Включает в себя возможность отслеживания 5 000 посещений, 4 страницы на сайте.<br><br>
2. Базовая версия. В этой версии можно отследить 10 000 посещений, и 10 страниц, что вполне достаточно для среднего корпоративного сайта. Стоит базовый комплект — 9 долларов в месяц. В этот комплект включены все дополнительные функции.<br><br>
3. Версия «Стандарт». В неё входит возможность отслеживания 25 000 посещений на 20 страницах. Вполне подходит для тестирования нового стартапа. Стоит она 19 долларов в месяц, именно её я купил для тестирования сервиса, и написания этого обзора.<br><br>
4. Версия «Плюс». Отличается от предыдущей возможностью отслеживания 100 000 посещений, 50 страниц. Очень хороший тариф для крупных сервисов. Стоит 49 долларов в месяц. Довольно большие деньги за сервис, но они обычно с лихвой окупаются.<br><br>
5. Версия «Про». Стоит почти 100 долларов, имеет возможность отследить 250 000 посещений на ста страницах. Тариф подходит для монстров с большой посещаемостью и большим количеством страниц.<br><br><img src="https://habrastorage.org/getpro/habr/post_images/f73/fb2/62d/f73fb262da4618a8dde67690cfd191ea.jpg" alt="Отслеживание статистики"></blockquote><br><blockquote><h3>Есть и аналоги</h3><br>
Я не поленился, и собрал еще пару ссылок с аналогами, которые предлагают такие же услуги, но немного дешевле.</blockquote><br><blockquote> 1. <a href="https://www.google.com/analytics/home/?hl=en" title="Шикарный сбор и анализ статистики">Google Analytics</a> — бесплатный сервис для сбора и анализа статистики, вывод статистики в наиболее наглядной форме, и без разнообразных рейтингов, счетчиков. Очень подробная и полезная вещь. Рекомендую, т.к. сам пользуюсь им для этого блога.<br><br>
2. <a href="http://www.mapsurface.com/" title="Сервис для отслеживания популярности блоков на сайте">MapSurface</a> — сам еще не использовал (т.к. предпочел CrazyEgg), но врядли будучи плохим сервис собрал бы множество положительных отзывов. К сожалению он сейчас находится в статусе закрытой беты.</blockquote><br><blockquote><h3>Вывод</h3><br>
Использовать можно, и нужно. Вот только тарифы довольно больно кусаются, но обычно эти деньги потом с лихвой отбиваются на повышении конвертации посетителей в деньги. Использовать сервис нужно для тестирования рекламных мест и удобства отдельных страниц, что помогает опять же повысить конвертацию. В общем и целом — <a href="http://ttp://www.crazyegg.com">полезная вещь</a> для каждого владельца сайтов, а для юзабилиста вообще практически обязательна. К счастью для людей, которые поиздержались деньгами в этом месяце — есть полезные аналоги.<br><br><img src="http://img241.imageshack.us/img241/2346/confettitnml5.jpg" alt="Интересная идея визуализации популярности - конфети"></blockquote><br><br>
Автор: <a href="http://www.birzool.com/" title="Я пишу о юзабилити веб интерфейсов">Ярослав Бирзул</a> (DezmASter).<br>
Источник: <a href="http://www.birzool.com/crazyegg/" title="Сумасшедшие яйца, первоисточник">Блог о юзабилити веб интерфейсов</a>.<br><br>
PS: Всех с прошедшим Новым годом! От всей души желаю вам всего самого-самого лучшего, чего вы желаете только в самых сокровенных мечтах. Удачно вам провести время.
'''

С помощью регулярных выражение очистите текст от тэгов (мета информацию внутри тэгов тоже удалите). В получившемся тексте точки не должны слипаться с началом следующего предложений, также не должно быть лишних пробелов.

In [3]:
def remove_tags(text):
    rem_tag = re.sub(r"<[^>]+>", "", text)
    rem_tag = ''.join(rem_tag.splitlines())
    rem_tag = re.sub(r'(?<=[.,?!])(?=[^\s])', r' ', rem_tag)
    return rem_tag

In [4]:
text = remove_tags(text)
text

'Сегодняшняя заметка будет о сервисе отслеживания активности пользователя — CrazyEgg. Я не знаю кому обязан сервис таким говорящим именем, но оно работает, и хорошо. Запоминается? Отлично! Что это такое? Как уже сказано выше это сервис для отслеживания перемещения пользователей по сайту — кто куда кликнул, какие ссылки наиболее популярные и тому подобная, разнородная информация для юзабилистов. Сервис позволяет отслеживать активность определенных пользователей, и выводить эти данные в различных формах: «инфракрасная» — где чем активнее область, тем она «теплее», салюты (чем активнее область, тем больше конфети), простой список с сортировкой по активности, колбы (чем заполненнее колба, тем активнее область), облака (на мой взгляд наиболее удобный вариант — совмещает в себе все остальные вместе взятые). Для кого это? Разумеется сервис изначально планировался как первый помощник для юзабилистов, но пользоваться им может каждый, у кого есть деньги — интуитивный интерфейс, и хороший дизайн 

## 2. Задание на выбор слов после "не " (1 балл)

Вытащите из очищенного текста все слова, которые идут после частицы "не" (случаи, когда "не" - часть предыдущего слова, не должны извлекаться), **кроме слов, начинающихся на гласную букву**. Сама часть "не " не должна извлекаться.

In [5]:
re.findall('\sне\s([^уеыаоэяиюУЕЫАОЭЯИЮ][а-яА-ЯЁё]+)', text)

['знаю',
 'поленился',
 'бесплатен',
 'сайтов',
 'нужно',
 'лишним',
 'нужно',
 'потеряет',
 'бесплатен',
 'поленился']

## 3. Задание на выбор глаголов (1 балл)

Попробуйте вытащить из очищенного текста как можно больше глаголов (минимум 40 с повоторениями), не перечисляя их в паттерне полностью. Можете токенизировать текст и использовать re.match, а можете использовать re.findall с \\b в начале и конце паттерна, чтобы не матчить части слов.

В результате не должно быть не глаголов. Также постарайтесь сделать паттерн максимально коротким.

In [6]:
verbs = re.findall('\\b\s(\w+(?:ся|аю|ит|ет|те|ить|ать|сь))\s\\b', text)
verbs

['будет',
 'знаю',
 'позволяет',
 'выводить',
 'вместе',
 'планировался',
 'пользоваться',
 'может',
 'дает',
 'популяризировался',
 'предполагает',
 'написать',
 'поленился',
 'потестировать',
 'отслеживать',
 'хватает',
 'тасовать',
 'спорить',
 'поможет',
 'поможет',
 'располагать',
 'зайдет',
 'будет',
 'будет',
 'начнет',
 'работает',
 'потеряет',
 'имеет',
 'отследить',
 'входит',
 'подходит',
 'отследить',
 'подходит',
 'пользуюсь',
 'находится',
 'отбиваются',
 'помогает',
 'повысить',
 'поиздержались',
 'желаю',
 'желаете']

In [7]:
len(verbs)

41

## 4. Pymorphy vs Mystem (2 балла)

Лемматизируйте очищенный текст с помощью mystem и pymorphy. Сравните результаты: есть ли разница, что лучше. 

Для пайморфи используйте токенизацию из mystem, чтобы исключить влияние токенизации на результат.

**Mystem:**

In [8]:
mystem = Mystem()

In [9]:
text_mystem = mystem.lemmatize(text)
text_mystem[:10]

['сегодняшний', ' ', 'заметка', ' ', 'быть', ' ', 'о', ' ', 'сервис', ' ']

**Pymorphy**

In [10]:
morph = MorphAnalyzer()
words_analized = [morph.parse(token) for token in word_tokenize(text)]

In [11]:
morph = MorphAnalyzer()
words_analized = [morph.parse(token) for token in word_tokenize(text)]

In [12]:
words_analized[:3]

[[Parse(word='сегодняшняя', tag=OpencorporaTag('ADJF femn,sing,nomn'), normal_form='сегодняшний', score=1.0, methods_stack=((DictionaryAnalyzer(), 'сегодняшняя', 493, 7),))],
 [Parse(word='заметка', tag=OpencorporaTag('NOUN,inan,femn sing,nomn'), normal_form='заметка', score=1.0, methods_stack=((DictionaryAnalyzer(), 'заметка', 8, 0),))],
 [Parse(word='будет', tag=OpencorporaTag('VERB,impf,intr sing,3per,futr,indc'), normal_form='быть', score=1.0, methods_stack=((DictionaryAnalyzer(), 'будет', 620, 12),))]]

In [13]:
[x[0].normal_form for x in words_analized][:10]

['сегодняшний',
 'заметка',
 'быть',
 'о',
 'сервис',
 'отслеживание',
 'активность',
 'пользователь',
 '—',
 'crazyegg']

В пайморфи не учитываются пробелы и мы сразу получаем много данных о слове (но знаки препинания учитываются). Что подходит лучше, наверное, зависит от задачи. Подробная информация может быть нужна не всегда, в таком случае оптимальнее выбрать mystem

## 5. Разбиение на предложения в Mystem (2 балла)

С помощью Mystem разделите текст из первого задания (очищенный) на предложения. Сравните разбиение с razdel.sentenize

**Mystem:**

In [14]:
# сохраняем текст на комп
f = open('text.txt', 'w')
f.write(text)
f.close()

In [15]:
import json
t = [json.loads(line) for line in open('text_parsed.txt')]

In [16]:
t[0][:10]

[{'analysis': [{'lex': 'сегодняшний', 'gr': 'A=им,ед,полн,жен'}],
  'text': 'Сегодняшняя'},
 {'text': ' '},
 {'analysis': [{'lex': 'заметка', 'gr': 'S,жен,неод=им,ед'}],
  'text': 'заметка'},
 {'text': ' '},
 {'analysis': [{'lex': 'будет', 'gr': 'ADV,прдк='},
   {'lex': 'быть', 'gr': 'V,нп=непрош,ед,изъяв,3-л'}],
  'text': 'будет'},
 {'text': ' '},
 {'analysis': [{'lex': 'о', 'gr': 'PR='},
   {'lex': 'о', 'gr': 'INTJ='},
   {'lex': 'о', 'gr': 'S,сокр=пр,мн'},
   {'lex': 'о', 'gr': 'S,сокр=пр,ед'},
   {'lex': 'о', 'gr': 'S,сокр=вин,мн'},
   {'lex': 'о', 'gr': 'S,сокр=вин,ед'},
   {'lex': 'о', 'gr': 'S,сокр=дат,мн'},
   {'lex': 'о', 'gr': 'S,сокр=дат,ед'},
   {'lex': 'о', 'gr': 'S,сокр=род,мн'},
   {'lex': 'о', 'gr': 'S,сокр=род,ед'},
   {'lex': 'о', 'gr': 'S,сокр=твор,мн'},
   {'lex': 'о', 'gr': 'S,сокр=твор,ед'},
   {'lex': 'о', 'gr': 'S,сокр=им,мн'},
   {'lex': 'о', 'gr': 'S,сокр=им,ед'}],
  'text': 'о'},
 {'text': ' '},
 {'analysis': [{'lex': 'сервис', 'gr': 'S,муж,неод=пр,ед'}],
  '

*PS - не поняла, как это превратить в предложение*

**Razdel:**

In [17]:
sent_razdel = list(sentenize(text))

In [18]:
sent_razdel[:10]

[Substring(0,
           84,
           'Сегодняшняя заметка будет о сервисе отслеживания активности пользователя — CrazyEgg.'),
 Substring(85,
           164,
           'Я не знаю кому обязан сервис таким говорящим именем, но оно работает, и хорошо.'),
 Substring(165, 178, 'Запоминается?'),
 Substring(179, 187, 'Отлично!'),
 Substring(188, 202, 'Что это такое?'),
 Substring(203,
           396,
           'Как уже сказано выше это сервис для отслеживания перемещения пользователей по сайту — кто куда кликнул, какие ссылки наиболее популярные и тому подобная, разнородная информация для юзабилистов.'),
 Substring(397,
           811,
           'Сервис позволяет отслеживать активность определенных пользователей, и выводить эти данные в различных формах: «инфракрасная» — где чем активнее область, тем она «теплее», салюты (чем активнее область, тем больше конфети), простой список с сортировкой по активности, колбы (чем заполненнее колба, тем активнее область), облака (на мой взгляд наибол

In [19]:
[sent.text for sent in sent_razdel[:10]]

['Сегодняшняя заметка будет о сервисе отслеживания активности пользователя — CrazyEgg.',
 'Я не знаю кому обязан сервис таким говорящим именем, но оно работает, и хорошо.',
 'Запоминается?',
 'Отлично!',
 'Что это такое?',
 'Как уже сказано выше это сервис для отслеживания перемещения пользователей по сайту — кто куда кликнул, какие ссылки наиболее популярные и тому подобная, разнородная информация для юзабилистов.',
 'Сервис позволяет отслеживать активность определенных пользователей, и выводить эти данные в различных формах: «инфракрасная» — где чем активнее область, тем она «теплее», салюты (чем активнее область, тем больше конфети), простой список с сортировкой по активности, колбы (чем заполненнее колба, тем активнее область), облака (на мой взгляд наиболее удобный вариант — совмещает в себе все остальные вместе взятые).',
 'Для кого это?',
 'Разумеется сервис изначально планировался как первый помощник для юзабилистов, но пользоваться им может каждый, у кого есть деньги — интуити

Razdel показался удобнее как минимум потому, что он не требует установки дополнительных файлов на компьютер, все реализуется в тетрадке. Плюс можно быстро вытащить чистый текст с помощью атрибута. Но для работы с большим кол-вом данных больше подойдет mystem

## 6*. Дизамбигуация в Mystem (2 балла)

*необязательное задание на 10 баллов

Одна из фичей майстема - контекстная дизамбигуация (т.е. определение правильного разбора в зависимости от контекста).

Запустите mystem на очищенном тексте с дизамбигуацией и без и проверьте есть ли какие-то различия.

**С дизамбигуацией:**

In [20]:
mystem_analyzer = Mystem(disambiguation=True)

In [21]:
text_dis = mystem_analyzer.analyze(text)
text_dis[:10]

[{'analysis': [{'lex': 'сегодняшний', 'wt': 1, 'gr': 'A=им,ед,полн,жен'}],
  'text': 'Сегодняшняя'},
 {'text': ' '},
 {'analysis': [{'lex': 'заметка', 'wt': 1, 'gr': 'S,жен,неод=им,ед'}],
  'text': 'заметка'},
 {'text': ' '},
 {'analysis': [{'lex': 'быть',
    'wt': 0.1492564349,
    'gr': 'V,нп=непрош,ед,изъяв,3-л'}],
  'text': 'будет'},
 {'text': ' '},
 {'analysis': [{'lex': 'о', 'wt': 0.9737875835, 'gr': 'PR='}], 'text': 'о'},
 {'text': ' '},
 {'analysis': [{'lex': 'сервис', 'wt': 1, 'gr': 'S,муж,неод=пр,ед'}],
  'text': 'сервисе'},
 {'text': ' '}]

**Без дизамбигуации:**

In [22]:
mystem_analyzer = Mystem(disambiguation=False)

In [23]:
text_no_dis = mystem_analyzer.analyze(text)
text_no_dis[:10]

[{'analysis': [{'lex': 'сегодняшний', 'wt': 1, 'gr': 'A=им,ед,полн,жен'}],
  'text': 'Сегодняшняя'},
 {'text': ' '},
 {'analysis': [{'lex': 'заметка', 'wt': 1, 'gr': 'S,жен,неод=им,ед'}],
  'text': 'заметка'},
 {'text': ' '},
 {'analysis': [{'lex': 'будет', 'wt': 0.8507435651, 'gr': 'ADV,прдк='},
   {'lex': 'быть', 'wt': 0.1492564349, 'gr': 'V,нп=непрош,ед,изъяв,3-л'}],
  'text': 'будет'},
 {'text': ' '},
 {'analysis': [{'lex': 'о', 'wt': 0.9737875835, 'gr': 'PR='},
   {'lex': 'о', 'wt': 0.02613852081, 'gr': 'INTJ='},
   {'lex': 'о',
    'wt': 7.389567447e-05,
    'gr': 'S,сокр=(пр,мн|пр,ед|вин,мн|вин,ед|дат,мн|дат,ед|род,мн|род,ед|твор,мн|твор,ед|им,мн|им,ед)'}],
  'text': 'о'},
 {'text': ' '},
 {'analysis': [{'lex': 'сервис', 'wt': 1, 'gr': 'S,муж,неод=пр,ед'}],
  'text': 'сервисе'},
 {'text': ' '}]

Отличие: без дизамбигуации несколько вариантов разбора предлога "о"